# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
! pip install ultralytics
! pip install opencv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 993.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 18.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [4]:
from ultralytics import YOLO
model = YOLO("/content/YOLO_Model.pt")

from google.colab.patches import cv2_imshow




## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [5]:
#https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
cap = cv.VideoCapture('/content/YOLO_Video.mp4')


## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [6]:
ret,frame=cap.read()
height=frame.shape[0]
width=frame.shape[1]
fps=cap.get(cv.CAP_PROP_FPS)
img_size=(width,height)
print(frame.shape)
print(height)
print(width)
print(fps)


(1080, 1920, 3)
1080
1920
29.97002997002997


## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [7]:
from ultralytics import solutions

In [35]:
output=cv.VideoWriter("output.avi", cv.VideoWriter_fourcc('M','J','P','G'), fps, img_size)

In [38]:
r1 = [(200, 400), (250, 404), (1080, 300), (20, 360)]

r2 = [(1000, 1850), (500, 300), (1080, 200), (20, 360)]

r3 = [(-220, 0), (500, 200), (600, 850), (20, 360)]

r4 = [(20, 700) ,(600, 1220), (250, 500), (20, 360)]

counter1 = solutions.ObjectCounter(
    view_img=True,
    reg_pts=r1,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,
)
counter2 = solutions.ObjectCounter(
    view_img=True,
    reg_pts=r2,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,
)
counter3 = solutions.ObjectCounter(
    view_img=True,
    reg_pts=r3,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,
)
counter4 = solutions.ObjectCounter(
    view_img=True,
    reg_pts=r4,
    names=model.names,
    draw_tracks=True,
    line_thickness=2,
)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    tracks = model.track(frame, persist=True, show=False)

    frame = counter1.start_counting(frame, tracks)
    frame = counter2.start_counting(frame, tracks)
    frame = counter3.start_counting(frame, tracks)
    frame = counter4.start_counting(frame, tracks)
    output.write(frame)



WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Polygon Counter Initiated.

0: 384x640 1 bus, 16 cars, 62.9ms
Speed: 3.4ms preprocess, 62.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 15 cars, 62.9ms
Speed: 3.2ms preprocess, 62.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 16 cars, 62.8ms
Speed: 3.0ms preprocess, 62.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 16 cars, 60.9ms
Speed: 3.1ms preprocess, 60.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 15 cars, 47.7ms
Speed: 3.1ms preprocess, 47.7ms i

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

## Save and Submit Your Work

Submit both the notebook and the output video